# 데이터셋 (상가)
### 처리 기준 : 노션 페이지 내 지정된 열
- 상가
    - 대규모 점포현황 / 시장및 마트 / 일반음식점

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/shinjh/Desktop/likelion/mid_project/dataset_life_infra")

In [2]:
## 함수 init (결측 위경도 추가용)
import pandas as pd
import requests
import json
from tqdm import tqdm
from geopy.geocoders import Nominatim

# Kakao API로 좌표 검색 함수
def get_location_kakao(address_data, kakao_api_key):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address_data}'
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        api_json = response.json()
        if api_json['documents']:
            address = api_json['documents'][0]['address']
            return float(address['y']), float(address['x'])  # 위도, 경도 반환
    return None, None

# Nominatim으로 좌표 검색 함수
def get_location_nominatim(address_data):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address_data)
    if location:
        return location.latitude, location.longitude  # 위도, 경도 반환
    return None, None

# 데이터프레임 업데이트 함수
def update_coordinates(df, kakao_api_key):
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # 이미 값이 있는 경우 건너뜀
        try:
            if not pd.isna(row['WGS84위도']) and not pd.isna(row['WGS84경도']):
                continue

            # 주소 정보 우선순위: 소재지지번주소 -> 소재지도로명주소
            address = row['소재지지번주소'] if not pd.isna(row['소재지지번주소']) else row['소재지도로명주소']

            # Kakao API 호출
            lat, lon = get_location_kakao(address, kakao_api_key)
            if lat is None or lon is None:
                try:
                    # Kakao API 실패 시 Nominatim 호출
                    lat, lon = get_location_nominatim(address)
                except:
                    print(f"can't get info : {address}")
                    continue

            # 결과 저장
            df.at[idx, 'WGS84위도'] = lat
            df.at[idx, 'WGS84경도'] = lon
        
        except:
            if not pd.isna(row['위도']) and not pd.isna(row['경도']):
                continue

            # 주소 정보 우선순위: 소재지지번주소 -> 소재지도로명주소
            address = row['소재지지번주소'] if not pd.isna(row['소재지지번주소']) else row['소재지도로명주소']

            # Kakao API 호출
            lat, lon = get_location_kakao(address, kakao_api_key)
            if lat is None or lon is None:
                try:
                    # Kakao API 실패 시 Nominatim 호출
                    lat, lon = get_location_nominatim(address)
                except:
                    print(f"can't get info : {address}")
                    continue

            # 결과 저장|
            df.at[idx, '위도'] = lat
            df.at[idx, '경도'] = lon
            

# Kakao API Key 설정
kakao_api_key = "20953ebae869efd8abdc3657eba800b6"  # 카카오 API 키를 여기에 입력

In [3]:
import re

def clean_address(address):
    """
    주소 문자열을 정리하여 올바르게 공백을 삽입하고 불필요한 공백을 제거합니다.
    """
    if pd.isna(address):
        return address  # 주소가 비어 있으면 그대로 반환

    # Step 1: 기본 공백 삽입
    # 행정구역 구분자 뒤에 공백 삽입 (도, 시, 구, 군, 읍, 면, 동, 리, 로, 길 등)
    address = re.sub(r"(시)(?!\s)", r"\1 ", address)

    # 숫자와 특수 기호(-)는 그대로 유지
    address = re.sub(r"(?<=\d)-(?=\d)", "-", address)

    # Step 4: 여러 공백 제거 및 앞뒤 공백 정리
    address = re.sub(r"\s+", " ", address).strip()

    return address

In [ ]:
## 상가

data_supermart = pd.read_csv("./시장및마트현황.csv", encoding='cp949')
data_restaurant_ilsan = pd.read_csv("./일반음식점현황_인허가_고양시.csv", encoding='cp949')
data_restaurant_bundang = pd.read_csv("./일반음식점현황_인허가_성남시.csv", encoding='cp949')
data_large_scale_mart_ilsan = pd.read_csv("./대규모점포현황_고양시.csv", encoding='cp949')
data_large_scale_mart_bundang = pd.read_csv("./대규모점포현황_성남시.csv", encoding='cp949')
data_large_scale_mart_bundang = pd.read_csv("./대규모점포현황_성남시.csv", encoding='cp949')

In [15]:
data_supermart_sort = data_supermart[["시설구분명", "소재지도로명주소", "소재지지번주소", "WGS84위도", "WGS84경도"]]
data_restaurant_ilsan_sort = data_restaurant_ilsan[["사업장명", "소재지도로명주소","소재지지번주소","위도","경도","영업상태명","폐업일자","업태구분명정보"]]
data_restaurant_bundang_sort = data_restaurant_bundang[["사업장명", "소재지도로명주소","소재지지번주소","위도","경도","영업상태명","폐업일자","업태구분명정보"]]
data_large_scale_mart_ilsan_sort = data_large_scale_mart_ilsan[["사업장명","영업상태명","폐업일자","소재지도로명주소","소재지지번주소","WGS84위도","WGS84경도","업태구분명정보"]]
data_large_scale_mart_bundang_sort = data_large_scale_mart_bundang[["사업장명","영업상태명","폐업일자","소재지도로명주소","소재지지번주소","WGS84위도","WGS84경도","업태구분명정보"]]

### 필터링 기준 (마트)
> 사용 열 : "시설구분명", "소재지도로명주소", "소재지지번주소", "WGS84위도", "WGS84경도"
- 필터링 (1) : 주소가 빈값이 아닌경우에 대해 우선선택 (소재지도로명주소, 소재지지번주소)
- 필터링 (2) : 분당, 일산 단어가 포함된 위치 우선적으로 선택
- 필터링 (3) : 위도,경도가 모두 빈칸인 행 Search, 주소가 이상한 곳 제거

In [ ]:
data_supermart_sort = data_supermart_sort[(data_supermart_sort['소재지지번주소'].notna()) | (data_supermart_sort['소재지도로명주소'].notna())]

In [60]:
# 필터링 (2)
bundang = data_supermart_sort[
    data_supermart_sort["소재지도로명주소"].combine_first(data_supermart_sort["소재지지번주소"]).str.contains("분당", na=False)
]

# 필터링 (3)
bundang = bundang[bundang['WGS84경도'].notna() & bundang['WGS84위도'].notna()].reset_index(drop=True)

print(bundang.shape)
bundang.head()

# bundang.to_csv("./dataset_life_infra_분당_시장및마트.csv", encoding='utf-8')

(89, 5)


,시설구분명,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,전통시장,경기도 성남시 분당구 불정로386번길 10 서현동 효자촌미래타운아파트,경기도 성남시 분당구 서현동 307,37.373416,127.138036
1,전통시장,경기도 성남시 분당구 불정로376번길 31,경기도 성남시 분당구 서현동 311,37.372255,127.138871
2,전통시장,경기도 성남시 분당구 불정로376번길 37 서현동,경기도 성남시 분당구 서현동 312-3,37.371629,127.139444
3,전통시장,경기도 성남시 분당구 내정로165번길 38,경기도 성남시 분당구수내동 32,37.376661,127.116239
4,전통시장,경기도 성남시 분당구 내정로 165번길 38,경기도 성남시 분당구 수내동 32,37.376661,127.116239


In [62]:
# 필터링 (2)
ilsan = data_supermart_sort[
    data_supermart_sort["소재지도로명주소"].combine_first(data_supermart_sort["소재지지번주소"]).str.contains("일산", na=False)
]

# 필터링 (3)
ilsan = ilsan[ilsan['WGS84경도'].notna() & ilsan['WGS84위도'].notna()].reset_index(drop=True)

print(ilsan.shape)
ilsan.head()

# ilsan.to_csv("./dataset_life_infra_일산_시장및마트.csv", encoding='utf-8')

(56, 5)


,시설구분명,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,로컬푸드직매장,경기도 고양시 일산동구 장항로 226-68,경기도 고양시 일산동구 장항동 594번지,37.642994,126.777651
1,로컬푸드직매장,경기도 고양시 일산서구 일산로 640,경기도 고양시 일산서구 대화동 2032-2번지,37.682180,126.758286
2,로컬푸드직매장,경기도 고양시 일산동구 태극로 8,경기도 고양시 일산동구 장항동 1753번지 킨텍스원시티,37.663592,126.749219
3,전통시장,경기도 고양시 일산서구 일청로 12번길 9,경기도 고양시 일산서구 일산동 617-20,37.685623,126.770826
4,로컬푸드직매장,경기도 고양시 일산동구 숲속마을1로 34,경기도 고양시 일산동구 풍동 349-13번지,37.663956,126.799602


### 필터링 기준 (식당)
> 사용 열 : "사업장명", "소재지도로명주소","소재지지번주소","위도","경도","영업상태명","폐업일자","업태구분명정보"
- 필터링 (1) : 주소가 빈값이 아닌경우에 대해 우선선택 (소재지도로명주소, 소재지지번주소)
- 필터링 (2) : 분당, 일산 단어가 포함된 위치 우선적으로 선택
- 필터링 (3) : 위도,경도가 모두 빈칸인 행 Search, 주소가 이상한 곳 제거

In [106]:
data_restaurant_ilsan_sort = data_restaurant_ilsan[["사업장명", "소재지도로명주소","소재지지번주소","위도","경도","영업상태명","인허가일자","폐업일자","업태구분명정보"]]
data_restaurant_bundang_sort = data_restaurant_bundang[["사업장명", "소재지도로명주소","소재지지번주소","위도","경도","영업상태명","인허가일자","폐업일자","업태구분명정보"]]

data_restaurant_ilsan_sort = data_restaurant_ilsan_sort[(data_restaurant_ilsan_sort['소재지지번주소'].notna()) | (data_restaurant_ilsan_sort['소재지도로명주소'].notna())]
data_restaurant_bundang_sort = data_restaurant_bundang_sort[(data_restaurant_bundang_sort['소재지지번주소'].notna()) | (data_restaurant_bundang_sort['소재지도로명주소'].notna())]

In [109]:
# 필터링 (2)
bundang = data_restaurant_bundang_sort[
    data_restaurant_bundang_sort["소재지도로명주소"].combine_first(data_restaurant_bundang_sort["소재지지번주소"]).str.contains("분당", na=False)
]

# 포맷설정
bundang['인허가일자'] = bundang["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
bundang['폐업일자'] = bundang["폐업일자"].apply(lambda x: x if pd.isna(x) or "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
update_coordinates(bundang, kakao_api_key)

# # 필터링 (3)
# bundang = bundang[bundang['경도'].notna() & bundang['위도'].notna()].reset_index(drop=True)

print(f"폐엉업 포함 : {bundang.shape}, 영업만 : {bundang[bundang['영업상태명']=='영업'].shape}")

/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/3787399861.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundang['인허가일자'] = bundang["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/3787399861.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundang['폐업일자'] = bundang["폐업일자"].apply(lambda x: x if pd.isna(x) or "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
  0%|          | 0/13877 [00:00<?, ?it/s]

 73%|███████▎  | 10141/13877 [00:05<00:02, 1267.66it/s]

can't get info : 경기도 성남시 분당구 수내동 32 105-6,7,8호 601동 
can't get info : 경기도 성남시 분당구 대장동 293 판교대장 도시개발지구 A5BL블럭로트 15동 식당 


 80%|███████▉  | 11063/13877 [00:14<00:12, 219.50it/s] 

can't get info : 경기도 성남시 분당구 대장동 225 판교대장도시개발지구 A7,8BL 블럭 로트 1층 식당 


 89%|████████▉ | 12379/13877 [00:19<00:06, 243.85it/s]

can't get info : 경기도 성남시 분당구 대장동 161 판교대장지구 A10BL 1층 


100%|██████████| 13877/13877 [00:20<00:00, 664.89it/s]

폐엉업 포함 : (13877, 9), 영업만 : (4639, 9)


,사업장명,소재지도로명주소,소재지지번주소,위도,경도,영업상태명,인허가일자,폐업일자,업태구분명정보
0,#그술집,"경기도 성남시 분당구 매화로37번길 16 (야탑동, 1층)",경기도 성남시 분당구 야탑동 383-8 1층,37.411688,127.133038,영업,2005-07-01,NaN,경양식
2,(내어릴적)풍금,경기도 성남시 분당구 금곡로11번길 6 1층 전체 (구미동),경기도 성남시 분당구 구미동 127-1번지 1층전체,37.344151,127.110432,폐업,2007-11-15,2008-08-01,까페
3,"(능이닭,능이오리)백숙,배달전문","경기도 성남시 분당구 황새울로312번길 20 (서현동,분당태성빌딩 지하1층 B124호)",경기도 성남시 분당구 서현동 265-6 분당태성빌딩 지하1층 B124호,37.383026,127.122074,폐업,2011-06-10,2021-04-29,한식
4,(대명)능이오복탕,"경기도 성남시 분당구 새마을로177번길 7-8, 1,2층 (율동)","경기도 성남시 분당구 율동 288 1,2층",37.384651,127.149335,영업,1993-08-24,NaN,한식
5,(선)속초코다리냉면,"경기도 성남시 분당구 동판교로177번길 25, 1층 106호 (삼평동, 판교 호반 ...",경기도 성남시 분당구 삼평동 740 판교 호반 써밋 플레이스 1층 106호,37.397321,127.113552,폐업,2013-06-18,2024-10-07,한식


In [110]:
# 필터링 (2)
ilsan = data_restaurant_ilsan_sort[
    data_restaurant_ilsan_sort["소재지도로명주소"].combine_first(data_restaurant_ilsan_sort["소재지지번주소"]).str.contains("일산", na=False)
]

# 포맷설정
ilsan['인허가일자'] = ilsan["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
ilsan['폐업일자'] = ilsan["폐업일자"].apply(lambda x: x if pd.isna(x) or "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
update_coordinates(ilsan, kakao_api_key)

# 필터링 (3)
ilsan = ilsan[ilsan['경도'].notna() & ilsan['위도'].notna()].reset_index(drop=True)


print(f"폐엉업 포함 : {ilsan.shape}, 영업만 : {ilsan[ilsan['영업상태명']=='영업'].shape}")
ilsan.head()

/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/1306382130.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ilsan['인허가일자'] = ilsan["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/1306382130.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ilsan['폐업일자'] = ilsan["폐업일자"].apply(lambda x: x if pd.isna(x) or "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
100%|██████████| 22090/22090 [00:15<00:00, 1466.68it/s

폐엉업 포함 : (22081, 9), 영업만 : (5721, 9)


,사업장명,소재지도로명주소,소재지지번주소,위도,경도,영업상태명,인허가일자,폐업일자,업태구분명정보
0,'파스타'와,"경기도 고양시 일산동구 고양대로 762, 110,111,112호 (중산동, 하늘빌딩 )","경기도 고양시 일산동구 중산동 1688번지 하늘빌딩 110,111,112호",37.681790,126.780769,폐업,2012-04-16,2016-01-05,경양식
1,(BF)비에프,"경기도 고양시 일산동구 무궁화로 20-18 403호 일부 (장항동,하임빌로데오)",경기도 고양시 일산동구 장항동 759번지 하임빌로데오빌딩 403호 일부,37.661213,126.767265,폐업,2010-05-19,2012-05-29,까페
2,"(DEAR,U)디얼유","경기도 고양시 일산동구 중앙로1275번길 38-31, 1층 106호 (장항동, 일산...",경기도 고양시 일산동구 장항동 766 라페스타A,37.660475,126.769613,폐업,2017-10-30,2021-10-13,기타
3,(GoGo)고고비어,"경기도 고양시 일산서구 대화로 136, 114호 (대화동, 대성프라자)",경기도 고양시 일산서구 대화동 863-3번지 대성프라자 114호,37.670271,126.735046,영업,2009-03-04,NaN,한식
4,(THE)뚜벅이족발,"경기도 고양시 일산동구 고봉로 626, 1층 일부호 (성석동)",경기도 고양시 일산동구 성석동 1233,37.712633,126.788593,폐업,2018-10-24,2022-06-30,한식


In [111]:
bundang.to_csv("./dataset_life_infra_분당_일반음식점_인허가일자포함.csv", encoding='utf-8')
ilsan.to_csv("./dataset_life_infra_일산_일반음식점_인허가일자포함.csv", encoding='utf-8')

### 필터링 기준 (대규모점포)
> 사용 열 : "사업장명","영업상태명","폐업일자","소재지도로명주소","소재지지번주소","WGS84위도","WGS84경도","업태구분명정보"
- 필터링 (1) : 주소가 빈값이 아닌경우에 대해 우선선택 (소재지도로명주소, 소재지지번주소)
- 필터링 (2) : 분당, 일산 단어가 포함된 위치 우선적으로 선택
- 필터링 (3) : 위도,경도가 모두 빈칸인 행 Search, 주소가 이상한 곳 제거

In [94]:
data_large_scale_mart_ilsan_sort = data_large_scale_mart_ilsan[["사업장명","영업상태명","인허가일자","폐업일자","소재지도로명주소","소재지지번주소","WGS84위도","WGS84경도","업태구분명정보"]]
data_large_scale_mart_bundang_sort = data_large_scale_mart_bundang[["사업장명","영업상태명","인허가일자","폐업일자","소재지도로명주소","소재지지번주소","WGS84위도","WGS84경도","업태구분명정보"]]

In [95]:
display(data_large_scale_mart_ilsan_sort)

,사업장명,영업상태명,인허가일자,폐업일자,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도,업태구분명정보
0,라 몬테 이탈리아노,영업개시전,20190122,NaN,"경기도 고양시 일산동구 연리지로 51, 라몬테이탈리아노 (장항동)",경기도 고양시 일산동구 장항동 1800번지 라몬테이탈리아노,NaN,NaN,그 밖의 대규모점포
1,GIFC MALL,영업개시전,20190123,NaN,경기도 고양시 일산서구 킨텍스로 240 (대화동),경기도 고양시 일산서구 대화동 2603번지,37.666098,126.750520,그 밖의 대규모점포
2,고양삼송현대헤리엇,영업개시전,20210702,NaN,"경기도 고양시 덕양구 삼송로 222, 고양삼송현대헤리엇 (삼송동)",경기도 고양시 덕양구 삼송동 293,37.652896,126.899229,그 밖의 대규모점포
3,고양 일산 호수공원 가로수길,영업개시전,20170825,NaN,경기도 고양시 일산서구 주엽로 80 (대화동),경기도 고양시 일산서구 대화동 1050번지 185호,37.665939,126.756403,그 밖의 대규모점포
4,스타필드 고양,영업개시전,20160425,NaN,"370, 동산동동",경기도 고양시 덕양구 동산동 370번지,37.646939,126.895611,복합쇼핑몰
...,...,...,...,...,...,...,...,...,...
68,현대프리젠트프라자,휴업처리,19970705,NaN,경기도 고양시 일산동구 중앙로 1171 (장항동),경기도 고양시일산동구 장항동 895호,37.651322,126.777517,그 밖의 대규모점포
69,뉴코아백화점,휴업처리,20040906,NaN,경기도 고양시 일산동구 중앙로 1206 (마두동),경기도 고양시일산동구 마두동 797호,37.654745,126.777019,대형마트
70,월마트슈퍼센터화정점,휴업처리,19970129,NaN,경기도 고양시 덕양구 백양로 79 (화정동),경기도 고양시덕양구 화정동 976호,37.632576,126.833368,대형마트
71,라페스타,휴업처리,20030721,NaN,NaN,경기도 고양시일산동구 장항동 766외 23필지호,NaN,NaN,그 밖의 대규모점포


In [96]:
display(data_large_scale_mart_bundang_sort)

,사업장명,영업상태명,인허가일자,폐업일자,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도,업태구분명정보
0,GS THE FRESH 분당이매역점,영업개시전,2023-11-08,NaN,"경기도 성남시 분당구 이매로 45, 이수프라자 (이매동)",경기도 성남시 분당구 이매동 115-2 이수프라자,37.394981,127.125889,그 밖의 대규모점포
1,일렉트로마트 판교점,영업개시전,20160321,NaN,"경기도 성남시 분당구 대왕판교로606번길 10 (백현동, 알파리움1단지)",경기도 성남시 분당구 백현동 531번지,37.395846,127.109063,전문점
2,돌고래시장,정상영업,19961220,NaN,경기도 성남시 분당구 내정로174번길 42 (수내동),경기도 성남시분당구 수내동 53호,37.371197,127.122076,시장
3,모란시장,정상영업,1974-10-18,NaN,경기도 성남시 중원구 둔촌대로69번길 8 (성남동),경기도 성남시중원구 성남동 4214호,37.430635,127.127498,그 밖의 대규모점포
4,미래타운제2종합시장,정상영업,19941202,NaN,경기도 성남시 분당구 불정로376번길 31 (서현동),경기도 성남시분당구 서현동 311호,37.372287,127.138910,그 밖의 대규모점포
...,...,...,...,...,...,...,...,...,...
104,금산시장,휴업처리,19940818,NaN,경기도 성남시 분당구 내정로166번길 43 (수내동),경기도 성남시분당구 수내동 63-1호,37.370287,127.122548,그 밖의 대규모점포
105,한양프라자서현점,휴업처리,19921210,NaN,경기도 성남시 분당구 중앙공원로31번길 42 (서현동),경기도 성남시분당구 서현동 89호,37.383006,127.125437,쇼핑센터
106,판교시장,휴업처리,19970328,NaN,경기도 성남시 분당구 내정로113번길 4 (정자동),경기도 성남시분당구 정자동 54번지 1호,37.370592,127.114771,그 밖의 대규모점포
107,상지시장,휴업처리,19940402,NaN,경기도 성남시 분당구 내정로165번길 38 (수내동),경기도 성남시분당구 수내동 32호,37.376257,127.116653,그 밖의 대규모점포


In [55]:
data_large_scale_mart_sort = data_large_scale_mart_sort[(data_large_scale_mart_sort['소재지지번주소'].notna()) | (data_large_scale_mart_sort['소재지도로명주소'].notna())]

In [100]:
# 필터링 (2)
ilsan = data_large_scale_mart_ilsan_sort[
    data_large_scale_mart_ilsan_sort["소재지도로명주소"].combine_first(data_large_scale_mart_ilsan_sort["소재지지번주소"]).str.contains("일산", na=False)
]

# 포맷설정
ilsan['인허가일자'] = ilsan["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
ilsan['소재지지번주소'] = ilsan['소재지지번주소'].apply(clean_address)
update_coordinates(ilsan, kakao_api_key)

# 필터링 (3)
ilsan = ilsan[ilsan['WGS84경도'].notna() & ilsan['WGS84위도'].notna()].reset_index(drop=True)

print(f"{ilsan.shape}")
ilsan.head()

/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/2028035168.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ilsan['인허가일자'] = ilsan["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/2028035168.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ilsan['소재지지번주소'] = ilsan['소재지지번주소'].apply(clean_address)
 72%|███████▏  | 39/54 [00:00<00:00, 265.32it/s]

can't get info : 경기도 고양시 일산서구 일산동 1561-1호


100%|██████████| 54/54 [00:05<00:00,  9.58it/s] 

can't get info : 경기도 고양시 일산동구 장항동 766외 23필지호
(52, 9)


,사업장명,영업상태명,인허가일자,폐업일자,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도,업태구분명정보
0,라 몬테 이탈리아노,영업개시전,2019-01-22,NaN,"경기도 고양시 일산동구 연리지로 51, 라몬테이탈리아노 (장항동)",경기도 고양시 일산동구 장항동 1800번지 라몬테이탈리아노,37.659901,126.754939,그 밖의 대규모점포
1,GIFC MALL,영업개시전,2019-01-23,NaN,경기도 고양시 일산서구 킨텍스로 240 (대화동),경기도 고양시 일산서구 대화동 2603번지,37.666098,126.750520,그 밖의 대규모점포
2,고양 일산 호수공원 가로수길,영업개시전,2017-08-25,NaN,경기도 고양시 일산서구 주엽로 80 (대화동),경기도 고양시 일산서구 대화동 1050번지 185호,37.665939,126.756403,그 밖의 대규모점포
3,일산 벨라씨타(Bella Citta),영업개시전,2016-08-26,NaN,"경기도 고양시 일산동구 강송로 33 (백석동, 일산요진와이시티)",경기도 고양시 일산동구 백석동 1237번지,37.642359,126.792682,쇼핑센터
4,레이킨스몰,정상영업,2010-08-26,NaN,경기도 고양시 일산서구 호수로 817 (대화동),경기도 고양시 일산서구 대화동 2602번지,37.667964,126.751387,그 밖의 대규모점포


In [101]:
ilsan.to_csv("./dataset_life_infra_일산_대규모점포_인허가일자포함.csv")

In [102]:
# 필터링 (2)
bundang = data_large_scale_mart_bundang_sort[
    data_large_scale_mart_bundang_sort["소재지도로명주소"].combine_first(data_large_scale_mart_bundang_sort["소재지지번주소"]).str.contains("분당", na=False)
]

# 포맷설정
bundang['인허가일자'] = bundang["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
bundang['소재지지번주소'] = bundang['소재지지번주소'].apply(clean_address)
update_coordinates(bundang, kakao_api_key)

# 필터링 (3)
bundang = bundang[bundang['WGS84경도'].notna() & bundang['WGS84위도'].notna()].reset_index(drop=True)

print(f"{bundang.shape}")
bundang.head()

/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/3642660413.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundang['인허가일자'] = bundang["인허가일자"].apply(lambda x: x if "-" in x else f"{x[:4]}-{x[4:6]}-{x[6:]}")
/var/folders/q1/q9p2kx9d60z96qf9jk4c2jnh0000gn/T/ipykernel_3591/3642660413.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundang['소재지지번주소'] = bundang['소재지지번주소'].apply(clean_address)
 79%|███████▉  | 64/81 [00:00<00:00, 68.49it/s]

can't get info : 경기도 성남시 분당구 구미동 10, 11번지 1 호


 88%|████████▊ | 71/81 [00:05<00:01,  9.78it/s]

can't get info : 경기도 성남시 분당구 서현동 호


100%|██████████| 81/81 [00:10<00:00,  7.94it/s]

can't get info : 경기도 성남시 분당구 서현동 호
(78, 9)


,사업장명,영업상태명,인허가일자,폐업일자,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도,업태구분명정보
0,GS THE FRESH 분당이매역점,영업개시전,2023-11-08,NaN,"경기도 성남시 분당구 이매로 45, 이수프라자 (이매동)",경기도 성남시 분당구 이매동 115-2 이수프라자,37.394981,127.125889,그 밖의 대규모점포
1,일렉트로마트 판교점,영업개시전,2016-03-21,NaN,"경기도 성남시 분당구 대왕판교로606번길 10 (백현동, 알파리움1단지)",경기도 성남시 분당구 백현동 531번지,37.395846,127.109063,전문점
2,돌고래시장,정상영업,1996-12-20,NaN,경기도 성남시 분당구 내정로174번길 42 (수내동),경기도 성남시 분당구 수내동 53호,37.371197,127.122076,시장
3,미래타운제2종합시장,정상영업,1994-12-02,NaN,경기도 성남시 분당구 불정로376번길 31 (서현동),경기도 성남시 분당구 서현동 311호,37.372287,127.138910,그 밖의 대규모점포
4,"테마폴리스 지상1,2층",정상영업,2005-05-27,NaN,"경기도 성남시 분당구 성남대로925번길 16 (야탑동,테마폴리스 지상1,2층)","경기도 성남시 분당구 야탑동 341번지 테마폴리스 지상1,2층",37.413026,127.127214,그 밖의 대규모점포


In [103]:
bundang.to_csv("./dataset_life_infra_분당_대규모점포_인허가일자포함.csv")